<a href="https://colab.research.google.com/github/NamanGautam0404/NG.pro/blob/main/AI_Role_Assignment_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import gdown
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [7]:
# Load the video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Define quadrants (adjust coordinates based on your video dimensions)
quadrants = {
    1: (0, 0, frame_width // 2, frame_height // 2),
    2: (frame_width // 2, 0, frame_width, frame_height // 2),
    3: (0, frame_height // 2, frame_width // 2, frame_height),
    4: (frame_width // 2, frame_height // 2, frame_width, frame_height)
}

# Initialize video writer for output
output_video_path = 'processed_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize variables for tracking
previous_positions = {color: None for color in colors_to_track}
events = []

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to HSV color space for better color detection
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Track balls of each color
    for color_name, color_value in colors_to_track.items():
        lower_bound, upper_bound = color_boundaries[color_name]
        lower_bound = np.array(lower_bound, dtype=np.uint8)
        upper_bound = np.array(upper_bound, dtype=np.uint8)

        # Mask for the color
        mask = cv2.inRange(hsv, lower_bound, upper_bound)
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # Find contours in the mask
        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Track each contour
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 100:  # Adjust threshold as needed
                # Get centroid of the contour
                center = centroid(cnt)
                if center:
                    cx, cy = center
                    cv2.circle(frame, (cx, cy), 5, color_value, -1)

                    # Determine current quadrant
                    current_quadrant = None
                    for q_num, q_coords in quadrants.items():
                        x1, y1, x2, y2 = q_coords
                        if x1 <= cx <= x2 and y1 <= cy <= y2:
                            current_quadrant = q_num
                            break

                    # Check for entry or exit
                    if previous_positions[color_name] is None:
                        previous_positions[color_name] = current_quadrant

                    prev_quadrant = previous_positions[color_name]
                    if prev_quadrant != current_quadrant:
                        if current_quadrant is not None:
                            events.append((cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0, current_quadrant, color_name, 'Entry'))
                        if prev_quadrant is not None:
                            events.append((cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0, prev_quadrant, color_name, 'Exit'))
                        previous_positions[color_name] = current_quadrant

                    # Overlay text
                    if current_quadrant is not None:
                        text = f"Q{current_quadrant} - {color_name}"
                        cv2.putText(frame, text, (cx - 20, cy - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_value, 2)

    # Write frame to output video
    output_video.write(frame)

# Release video capture and writer
cap.release()
output_video.release()

# Write events to text file
output_txt_path = 'output_events.txt'
with open(output_txt_path, 'w') as f:
    for event in events:
        event_time, quadrant, ball_color, event_type = event
        f.write(f"Time: {event_time:.2f}s, Quadrant: {quadrant}, Ball Colour: {ball_color}, Event Type: {event_type}\n")

print("Processing completed.")
print(f"Processed Video saved as: {output_video_path}")
print(f"Event Data saved as: {output_txt_path}")

Processing completed.
Processed Video saved as: processed_video.avi
Event Data saved as: output_events.txt
